<a href="https://colab.research.google.com/github/elinoyam/FindMyParking/blob/WorkingWithColab/Calculate_Parking_Probability_DL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np


We want a model that get the number of free parking spots and occupied parking spots in a street from the bashini project and predicts the probability to find an empty parking spot in the street.

In [8]:
# linear regression model
# Create the model class
class ParkingProbModel(torch.nn.Module):
    def __init__(self,input_dim,output_dim): 
        super(ParkingProbModel, self).__init__()
        # One input and one output 
        self.linear = torch.nn.Linear(input_dim,output_dim)    

    def forward(self, x): 
        prediction = self.linear(x) 
        return prediction 

In [17]:
# our data is in the form: [...[free_parkings,occupied_parkings,probability]...]
# need to be transform to:
  # input_data = [...[free_parkings,occupied_parkings]...]
  # targets = [...,[probability],...]

def get_transformed_datasets(original_data):
  data = torch.tensor(original_data) # maybe needs to be torch.from_numpy()
  input_data = data[:,:2]
  labels = data[:,2:]
  
  return input_data, labels

In [ ]:
# assuming arr is the data we want the model to learn from
arr = [[1,2,3],[4,5,6],[7,8,9]]
validation_ratio = 0.2
batch_size = 16
input_dim = 2
output_dim = 1
epoch = 500

input_data, labels = get_transformed_datasets(arr)

dataset = TensorDataset(input_data, labels)

train_dataset, val_dataset = train_test_split(dataset, test_size=validation_ratio, shuffle=True)

dl_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
dl_valid = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Initializing the model
model = ParkingProbModel(input_dim, output_dim)
# Define the loss
loss_function = nn.CrossEntropyLoss()  
lr = 0.01 # if not good try the original lr: 0.001

# Initialize the optimizer
optimizers = torch.optim.SGD(model.parameters(), lr=lr)  

In [ ]:
# batchsiz = 100
# niter = 2000
# numepoch = niter / (len(trainds) / batchsiz)
# numepoch = int(numepoch)

# Train the model
iters = 0
for epoch in range(epoch):
    for x, (params, labels) in enumerate(dl_train):
        optimizers.zero_grad() # clear the gradient
        
        prediction = model(params) # Forward pass to get output

        losses = loss_function(prediction, labels)

        losses.backward()

        optimizers.step()

        iters += 1

        if iters % 50 == 0: # check status
            corrects = 0
            totals = 0
            
            for inputs, lbls in dl_valid:
                outps = model(inputs)

                _, predict = torch.max(outps.data, 1)

                totals += lbls.size(0)
                corrects += (predict == lbls).sum() # total number of correct prediction.

            accuracy = 100 * corrects / totals

            print('Iterations: {}. Loss: {}. Accuracy: {}'.format(iters, losses.item(), accuracy))
